In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from IPython.display import HTML
from src.config import ASSETS_ABSPATH
from src.utils import apply_vbt_settings

apply_vbt_settings()
pd.set_option("display.max_rows", None)

HTML(f"{ASSETS_ABSPATH}/vscode-jupyter-styles.html")

### Range Splitting

In [20]:
import sys

if sys.modules.get("checks") is not None:
    del sys.modules["checks"]
from checks import Checks

c = Checks("USD", ["BTC"])
c.get_data(3, "1m", split={"n": 100, "range_len": 60 * 24})
hp = c.run_hypothesis(
    close_positions=True,
    rsi_window=np.arange(7, 21, 3, dtype=int),
    rsi_entry=np.arange(20, 40, 5, dtype=int),
    rsi_exit=np.arange(60, 90, 5, dtype=int),
)
pf = c.create_portfolio(
    entries=hp.entry_signals, exits=hp.exit_signals, freq="1T", fees=0.001
)
pf

Pulling new data…


In [21]:
fig = pf.total_return().vbt.volume(
    x_level="hp_rsi_exit",
    y_level="hp_rsi_entry",
    z_level="hp_rsi_window",
    slider_level="split_idx",
)
fig.show()

In [24]:
mean = (
    pf.total_return()
    .groupby(level=["hp_rsi_exit", "hp_rsi_entry", "split_idx"])
    .mean()
)
fig = mean.vbt.heatmap(
    x_level="hp_rsi_entry", y_level="hp_rsi_exit", slider_level="split_idx"
)
fig.show()

### Box Plot

In [47]:
import sys

if sys.modules.get("checks") is not None:
    del sys.modules["checks"]
from checks import Checks

PF_COMMON = {"freq": "1T", "fees": 0.001}

c = Checks("USD", ["BTC"])
c.get_data(5, "1m", split={"n": 100, "range_len": 60 * 24})
hp = c.run_hypothesis(
    close_positions=True,
    rsi_window=50
    # rsi_window=np.arange(7, 21, 3, dtype=int),
    # rsi_entry=np.arange(20, 40, 5, dtype=int),
    # rsi_exit=np.arange(60, 90, 5, dtype=int),
)
hp_pf = c.create_portfolio(
    entries=hp.entry_signals, exits=hp.exit_signals, **PF_COMMON
)
ct = c.run_control()
ct_pf = c.create_portfolio(
    entries=ct.entry_signals, exits=ct.exit_signals, **PF_COMMON
)
hp_pf

Pulling new data…


In [48]:
com_pf_df = pd.DataFrame({
    "hp": list(hp_pf.total_return()),
    "ct": list(ct_pf.total_return()),
})
comp_box = vbt.plotting.Box(data=com_pf_df, trace_names=["hp", "ct"])
comp_box.fig.show()